In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(palette='pastel')
sns.set(font_scale=2)

# Abstract
## 1. pre process
### 1) data load, drop duplicated
### 2) finiding real value
### 3) seperating exchange and one-way trade

## 2. 변수 추가
### 1) 몇번 trade
#### source_cnt_one,target_cnt_one,exchange_cnt
### 2) 몇명과 trade
#### from_how_many_one,to_how_many_one,with_how_many
### 3) 얼마나 많은 사람과(에게) gini impurity
#### too_gini_one,from_gini_one,with_gini,give_category_sum_one
### 4) 어떤 category를 얼마나(sum)
#### give_category_sum_one,receive_category_sum_one,give_categoty_sum_exc,receive_category_sum_exc
### 5) category별 비율(count 기준)
#### give_category_ratio_onereceive_category_ratio_one,give_category_ratio_exc,receive_category_ratio_exc
### 6) 마지막 trade
#### give_last_week,receive_last_week,exc_last_week,
### 7) 가장 많이 trade한 week
#### give_most_weeks,receive_most_weeks,exc_most_weeks

## 3. Merging data
## 4. 관련 issue

# Preprocessing

## 1.데이터 불러오기, 중복값 제거

In [3]:
test_data=pd.read_csv('transformed/test_real_datas_int.csv')
train_data_with_label=pd.read_csv('transformed/real_datas_int_num_id_label.csv')
train_data=pd.read_csv('transformed/real_values_int.csv')

In [4]:
train_trade=pd.read_csv('lite_data/train_trade_lite.csv')
test_trade=pd.read_csv('lite_data/test_trade_lite.csv')

In [168]:
train_trade=train_trade.drop_duplicates()
train_trade.head()

,trade_week,trade_day,trade_time,source_acc_id,target_acc_id,item_type,item_amount
0,8,2,23:32:06,tr58254,ot95753,grocery,-0.056344
1,7,6,05:06:31,tr58254,ot81514,grocery,-0.056343
2,1,2,20:05:18,te36067,tr94292,grocery,-0.056346
3,1,2,22:18:09,te36067,tr94292,grocery,-0.056346
4,1,3,00:31:10,te36067,tr94292,grocery,-0.056346


In [169]:
len(train_trade)

5542854

In [170]:
test_trade=test_trade.drop_duplicates()
test_trade.head()

,trade_week,trade_day,trade_time,source_acc_id,target_acc_id,item_type,item_amount
0,6,7,00:45:44,te36067,ot78976,grocery,-0.056346
1,6,7,02:06:56,te36067,ot78976,grocery,-0.056346
2,6,7,17:07:37,te36067,ot78976,grocery,-0.056346
3,6,7,18:56:27,te36067,ot78976,grocery,-0.056346
4,7,2,18:59:17,te36067,ot78976,grocery,-0.056345


## 2.real_value 찾기
### 2.1 찾을 수 있는지 확인, 찾기

In [171]:
#amount의 value_count
train_trade.item_amount.value_counts().sort_index()

-0.056346      208298
-0.056346       64101
-0.056346       39535
-0.056346       34061
-0.056346       77565
-0.056346       60324
-0.056346       19684
-0.056346       17240
-0.056346       35272
-0.056346       30230
-0.056346       15325
-0.056346       23629
-0.056346       75568
-0.056346       40055
-0.056346       33971
-0.056346       27973
-0.056346       25073
-0.056346       30008
-0.056346       46655
-0.056346      181754
-0.056346      241819
-0.056346      126683
-0.056346       69038
-0.056346       34226
-0.056346       22388
-0.056346       21926
-0.056346       23521
-0.056346       25214
-0.056346       25467
-0.056346       24304
                ...  
 121.558400         2
 122.427077         1
 123.295754         1
 124.946240         1
 125.901784         2
 127.237722         1
 128.499128         1
 130.245168         6
 131.587273         2
 135.370361         1
 136.325905         1
 138.063259         1
 138.098006         1
 138.931935         1
 140.66928

In [172]:
concated=pd.concat([train_trade,test_trade])

In [173]:
concated=concated.drop_duplicates()

In [174]:
#원래 값으로 바꿔주는 함수, trade의 속성에 맞게 변환
def to_real_data_with_different_format(dat,cnames,digit_precision=5,depth=1,as_int=False,with_label=False):
    res=[]
    colname_of_res=[]
    np.set_printoptions(precision=digit_precision)
    np.set_printoptions(suppress=True)
    for cname in cnames:
        temp=dat[cname].copy()
        unique_values=temp.unique()
        unique_values.sort()
        for i in range(depth):
            if i == 0:
                diffs=np.diff(unique_values)
            else:
                diffs=np.diff(diffs)
                diffs=diffs[diffs!=0]
            if as_int==True:
                res.append((dat[cname]/min(diffs)-(dat[cname]/min(diffs)).min()).round().astype(int))
            else:
                res.append(dat[cname]/min(diffs)-(dat[cname]/min(diffs)).min())
            if depth == 1:
                colname_of_res.append(cname)
            else:
                colname_of_res.append(cname+'_depth_'+str(i+1))
    res=np.array(res).transpose()
    res=pd.DataFrame(res,columns=colname_of_res)
    return res



In [175]:
#depth1으로 변환
real_trade=to_real_data_with_different_format(concated,['item_amount'])

In [176]:
#값 확인
real_trade.item_amount.value_counts()

2.000015e+01    286766
0.000000e+00    272602
1.900015e+01    206179
2.100016e+01    150392
1.000007e+06    134343
4.000031e+00    100804
1.200009e+01     89958
2.200017e+01     83786
4.100031e+01     75861
1.000008e+00     75823
4.000031e+01     74833
5.000038e+00     74094
3.900030e+01     61316
4.200032e+01     59203
1.800014e+01     57329
5.000037e+06     55680
2.000015e+00     49510
1.300010e+01     47445
8.000061e+00     46171
3.000023e+00     43734
2.300018e+01     41133
3.800029e+01     41123
1.400011e+01     41011
3.000022e+06     39687
4.300033e+01     38547
9.000069e+00     37885
1.700013e+01     36925
1.500011e+01     33773
2.800021e+01     31563
1.000008e+07     31429
                 ...  
9.855074e+06         1
1.582942e+05         1
6.596880e+05         1
2.041416e+08         1
7.643038e+05         1
2.791631e+05         1
3.369486e+05         1
4.472484e+05         1
6.836422e+05         1
9.945316e+05         1
1.022945e+06         1
9.231031e+05         1
9.183290e+0

### 2.2 제대로 변환이 안된거 같으니 다시 나눌 값을 찾아봄

In [177]:
#반복되는 페턴을 탐색
concated.item_amount.diff().value_counts().index[:40]

Float64Index([                    0.0,   -8.68676759691045e-09,
                 8.68676759691045e-09,  -8.686767603849344e-09,
                8.686767603849344e-09, -1.7373535200759793e-08,
               1.7373535200759793e-08,  -8.686767499765935e-09,
                8.686767499765935e-09,  -4.343383790128552e-08,
                4.343383790128552e-08, -2.6060302700525728e-08,
               -3.474707029743618e-08,  2.6060302700525728e-08,
                3.474707029743618e-08,  -1.737353510361528e-08,
                1.737353510361528e-08, -2.6060302797670243e-08,
               2.6060302797670243e-08,  -5.212060549819597e-08,
                5.212060549819597e-08, -1.7373535096676385e-08,
               1.7373535096676385e-08,   9.555444339948149e-08,
                6.080737310204531e-08,  -6.080737310204531e-08,
               -9.555444339948149e-08,  -3.474707030437507e-08,
                6.949414069895576e-08,  -6.949414069895576e-08,
                3.474707030437507e-08,  

In [178]:
#하나도 안주는 것은 말이 안되니 minimum 값 +1을하겠음
rest=(((concated.item_amount/0.0000000086867675969)-(concated.item_amount/0.0000000086867675969).min()))+1

In [179]:
train_trade.item_type.value_counts().index

Index(['money', 'grocery', 'costume', 'accessory', 'gem', 'weapon'], dtype='object')

In [180]:
#grocery의 경우 변환이 잘 되었는지 확인
sum(((rest[concated.item_type=='grocery']%1)>0.00)&((rest[concated.item_type=='grocery']%1)<0.99))

0

In [181]:
#값을 확인해보니 반올림해도 되는 것을 확인
rest[concated.item_type=='grocery'].value_counts()

21.000000       286743
1.000000        268999
20.000000       206152
22.000000       150371
5.000000        100752
13.000000        89938
23.000000        83773
42.000000        75848
2.000000         75479
41.000000        74811
6.000000         74050
40.000000        61298
43.000000        59185
19.000000        57313
3.000000         49334
14.000000        47428
9.000000         46144
4.000000         43650
24.000000        41113
39.000000        41101
15.000000        40989
44.000000        38536
10.000000        37813
18.000000        36893
16.000000        33750
29.000000        31549
28.000000        30942
17.000000        30769
30.000000        30103
38.000000        29915
                 ...  
20695.999967         1
7433.999988          1
15400.999976         1
60984.999904         1
11487.999982         1
45938.999927         1
7460.999988          1
3852.999994          1
30816.999951         1
6317.999990          1
53762.999915         1
17399.999973         1
4941.999992

In [182]:
rest[concated.item_type=='grocery']=rest[concated.item_type=='grocery'].round()

In [183]:
#gem의 경우 변환이 잘되었는지 확인
sum(((rest[concated.item_type=='gem']%1)>0.00)&((rest[concated.item_type=='gem']%1)<0.99))

0

In [184]:
#gem의 경우도 반올림해서 사용해도 됨
rest[concated.item_type=='gem'].value_counts()

1.0    662
2.0     41
3.0     20
4.0      8
6.0      6
8.0      5
5.0      4
7.0      4
Name: item_amount, dtype: int64

In [185]:
rest[concated.item_type=='gem']=rest[concated.item_type=='gem'].round()

In [186]:
#costume의 경우 변환이 잘되었는지 확인
sum(((rest[concated.item_type=='costume']%1)>0.00)&((rest[concated.item_type=='costume']%1)<0.99))

0

In [187]:
#costume의 경우도 반올림해서 사용해도 됨
rest[concated.item_type=='costume'].value_counts()

1.0    1304
2.0      99
3.0      53
4.0      19
5.0       7
7.0       5
8.0       1
6.0       1
Name: item_amount, dtype: int64

In [188]:
rest[concated.item_type=='costume']=rest[concated.item_type=='costume'].round()

In [189]:
#grocery의 경우 변환이 잘 되었는지 확인
sum(((rest[concated.item_type=='accessory']%1)>0.00)&((rest[concated.item_type=='accessory']%1)<0.99))

0

In [190]:
#gem의 경우도 반올림해서 사용해도 됨
rest[concated.item_type=='accessory'].value_counts()

1.0     989
2.0     143
3.0      28
4.0      21
5.0      14
6.0       5
10.0      3
7.0       1
8.0       1
Name: item_amount, dtype: int64

In [191]:
rest[concated.item_type=='accessory']=rest[concated.item_type=='accessory'].round()

In [192]:
#weapon의 경우 변환이 잘 되었는지 확인
sum(((rest[concated.item_type=='weapon']%1)>0.00)&((rest[concated.item_type=='weapon']%1)<0.99))

0

In [193]:
#weapon의 경우도 반올림해서 사용해도 됨
rest[concated.item_type=='weapon'].value_counts()

1.0     329
2.0      21
3.0       7
5.0       5
14.0      3
7.0       3
6.0       2
26.0      1
30.0      1
10.0      1
11.0      1
52.0      1
20.0      1
13.0      1
21.0      1
4.0       1
Name: item_amount, dtype: int64

In [194]:
rest[concated.item_type=='weapon']=rest[concated.item_type=='weapon'].round()

In [195]:
#money의 경우 변환이 잘 되었는지 확인
sum(((rest[concated.item_type=='money']%1)>0.00)&((rest[concated.item_type=='money']%1)<0.99))

371231

In [196]:
#money의 경우는 더 살펴봐야 함
rest[concated.item_type=='money'].value_counts()

1.000000e+06    134343
5.000000e+06     55680
3.000000e+06     39687
1.000000e+07     31429
2.000000e+06     25689
2.000000e+07     15277
3.000000e+07     14687
5.000000e+07     12145
4.000000e+06     12007
1.000000e+08     11674
6.000000e+06     11230
4.000000e+07      8700
7.000000e+06      6428
1.500000e+07      6266
5.000000e+05      4934
8.000000e+06      4765
6.000000e+07      4599
2.000000e+08      4450
7.000000e+07      4174
8.000000e+07      3652
1.800000e+07      3632
9.000000e+06      3607
1.500000e+06      3230
1.100000e+07      3137
5.370000e+03      2928
2.500000e+07      2874
3.000000e+08      2874
1.200000e+07      2737
1.500000e+08      2627
1.300000e+07      2375
                 ...  
7.385880e+05         1
6.005210e+05         1
4.323720e+05         1
7.059510e+05         1
3.705490e+05         1
3.099590e+05         1
2.262610e+05         1
1.680711e+06         1
1.340000e+04         1
9.345790e+05         1
5.263230e+05         1
8.028360e+05         1
9.862820e+0

In [197]:
rest[concated.item_type=='money']=rest[concated.item_type=='money'].round()

In [198]:
concated.item_amount=rest

In [199]:
concated.head()

,trade_week,trade_day,trade_time,source_acc_id,target_acc_id,item_type,item_amount
0,8,2,23:32:06,tr58254,ot95753,grocery,219.0
1,7,6,05:06:31,tr58254,ot81514,grocery,302.0
2,1,2,20:05:18,te36067,tr94292,grocery,27.0
3,1,2,22:18:09,te36067,tr94292,grocery,42.0
4,1,3,00:31:10,te36067,tr94292,grocery,29.0


In [200]:
train_trade.item_amount=(((train_trade.item_amount/0.0000000086867675969)-(train_trade.item_amount/0.0000000086867675969).min()))+1
train_trade.item_amount=train_trade.item_amount.round()

In [201]:
test_trade.item_amount=(((test_trade.item_amount/0.0000000086867675969)-(test_trade.item_amount/0.0000000086867675969).min()))+1
test_trade.item_amount=test_trade.item_amount.round()

In [376]:
#저장 및 불러오기
#train_trade.to_csv('transformed/train_trade_float.csv',index=False)
train_trade=pd.read_csv('transformed/train_trade_float.csv')

In [377]:
#저장 및 불러오기
#test_trade.to_csv('transformed/test_trade_float.csv',index=False)
test_trade=pd.read_csv('transformed/test_trade_float.csv')

In [378]:
concated=pd.concat([train_trade,test_trade])
concated=concated.drop_duplicates()

In [379]:
concated.index=range(len(concated))

In [383]:
concated.head()

,trade_week,trade_day,trade_time,source_acc_id,target_acc_id,item_type,item_amount
0,8,2,23:32:06,tr58254,ot95753,grocery,219.0
1,7,6,05:06:31,tr58254,ot81514,grocery,302.0
2,1,2,20:05:18,te36067,tr94292,grocery,27.0
3,1,2,22:18:09,te36067,tr94292,grocery,42.0
4,1,3,00:31:10,te36067,tr94292,grocery,29.0


## 3. seperating exchange and one-way trade

In [381]:
multiple_transactions=concated.copy()
multiple_transactions=multiple_transactions.sort_values(['trade_week','trade_day','trade_time'])
multiple_transactions.index=range(len(multiple_transactions))

In [384]:
multiple_transactions.head(30)

,trade_week,trade_day,trade_time,source_acc_id,target_acc_id,item_type,item_amount
0,1,1,00:00:00,ot51070,tr88265,grocery,2.0
1,1,1,00:00:00,ot51070,tr88265,money,285122.0
2,1,1,00:00:00,ot127363,te1423,money,2200000.0
3,1,1,00:00:02,tr39831,ot97116,grocery,1.0
4,1,1,00:00:02,tr39831,ot97116,money,285134.0
5,1,1,00:00:02,ot155752,tr13414,money,2000000.0
6,1,1,00:00:02,te874,ot65275,grocery,3.0
7,1,1,00:00:02,te874,ot65275,money,414474.0
8,1,1,00:00:03,tr91831,tr36321,money,9917600.0
9,1,1,00:00:03,ot61220,te17065,grocery,29.0


In [334]:
mults=multiple_transactions.groupby(['trade_week','trade_day','trade_time','source_acc_id','target_acc_id'],as_index=False).count()
source_same_time=[]
target_same_time=[]
crr_times=[]
source_group=[]
target_group=[]
for i in range(len(mults)):
    if i==0:
        current_timestamp=mults.loc[i,['trade_week','trade_day','trade_time']]
        source_group.append(mults.source_acc_id[i])
        target_group.append(mults.target_acc_id[i])
        crr_times.append(current_timestamp)
    else:
        if sum(mults.loc[i,['trade_week','trade_day','trade_time']]==current_timestamp)!=3:
            crr_times.append(current_timestamp)
            current_timestamp=mults.loc[i,['trade_week','trade_day','trade_time']]
            source_same_time.append(source_group)
            target_same_time.append(target_group)
            source_group=[]
            target_group=[]
            source_group.append(mults.source_acc_id[i])
            target_group.append(mults.target_acc_id[i])
        else:
            source_group.append(mults.source_acc_id[i])
            target_group.append(mults.target_acc_id[i])

In [389]:
crr_times[1]

trade_week           1
trade_day            1
trade_time    00:00:00
Name: 0, dtype: object

In [387]:
source_same_time[9]

['tr10943', 'tr10989', 'tr75488']

In [390]:
exchanges_list=[]
for i in range(len(source_same_time)):
    for j in source_same_time[i]:
        if j in target_same_time[i]:
            temp=list(crr_times[i+1])
            temp.append(j)
            exchanges_list.append(temp)


In [391]:
exchanges_df=pd.DataFrame(exchanges_list)
exchanges_df.columns=['trade_week','trade_day','trade_time','source_acc_id']
exchanges_df['is_exchange']=1
exchanges_df.head()

,trade_week,trade_day,trade_time,source_acc_id,is_exchange
0,1,1,00:00:10,tr10943,1
1,1,1,00:00:10,tr75488,1
2,1,1,00:00:14,te7050,1
3,1,1,00:00:14,tr8965,1
4,1,1,00:00:23,ot65557,1


In [393]:
multiple_transactions_final=pd.merge(multiple_transactions,exchanges_df,on=['trade_week','trade_day','trade_time','source_acc_id'], how='left')

In [401]:
multiple_transactions_final.iloc[:,-1]=multiple_transactions_final.iloc[:,-1].fillna(0)
multiple_transactions_final.to_csv('Features/trade_with_exchange_bool.csv',index=False)
#multiple_transactions_final=pd.read_csv('Features/trade_with_exchange_bool.csv')

In [ ]:
# exchange를 분리

In [397]:
exchanges=multiple_transactions_final[multiple_transactions_final['is_exchange']==1].iloc[:,:-1]

In [398]:
exchanges.head()

,trade_week,trade_day,trade_time,source_acc_id,target_acc_id,item_type,item_amount
22,1,1,00:00:10,tr75488,tr10943,grocery,20.0
24,1,1,00:00:10,tr10943,tr75488,money,1818.0
29,1,1,00:00:14,tr8965,te7050,grocery,1288.0
30,1,1,00:00:14,te7050,tr8965,money,1.0
39,1,1,00:00:23,tr60874,ot65557,grocery,5.0


### 3.1. concated에서 exchange data를 뺀 데이터 생성


In [403]:
except_exchange=multiple_transactions_final[multiple_transactions_final.is_exchange==0].iloc[:,:-1]
except_exchange.head()

,trade_week,trade_day,trade_time,source_acc_id,target_acc_id,item_type,item_amount
0,1,1,00:00:00,ot51070,tr88265,grocery,2.0
1,1,1,00:00:00,ot51070,tr88265,money,285122.0
2,1,1,00:00:00,ot127363,te1423,money,2200000.0
3,1,1,00:00:02,tr39831,ot97116,grocery,1.0
4,1,1,00:00:02,tr39831,ot97116,money,285134.0


In [406]:
#잘 되었는지 확인
len(multiple_transactions_final)-len(exchanges)-len(except_exchange)

0

# 변수 추가

## 1.몇번 줬는가, 몇번 받았는가

  ###   1.1.일방향 몇번 주고 받았는가(source_cnt_one,target_cnt_one)

In [413]:
#source id 별 언급된 회수
source_cnt_one=except_exchange.groupby(['trade_week','trade_day','trade_time','source_acc_id'],as_index=False).count().groupby('source_acc_id',as_index=False).count().iloc[:,:2]
source_cnt_one.head()

,source_acc_id,trade_week
0,ot100002,1
1,ot100006,7
2,ot100007,1
3,ot10001,1
4,ot100013,113


In [415]:
target_cnt_one=exchanges.groupby(['trade_week','trade_day','trade_time','target_acc_id'],as_index=False).count().groupby('target_acc_id',as_index=False).count().iloc[:,:2]
target_cnt_one.head()

,target_acc_id,trade_week
0,ot100007,3
1,ot100009,4
2,ot100013,81
3,ot100020,1
4,ot10003,4


### 1.2. 몇번 교환했는가(exchange_cnt)

In [417]:
#id별 교환 횟수
exchange_cnt=exchanges.groupby(['trade_week','trade_day','trade_time','source_acc_id'],as_index=False).count().groupby('source_acc_id',as_index=False).count().iloc[:,:2]
exchange_cnt.columns=['id','exchange_cnt']
exchange_cnt.head(10)

,id,exchange_cnt
0,ot100007,3
1,ot100009,4
2,ot100013,81
3,ot100020,1
4,ot10003,4
5,ot100039,1
6,ot100045,4
7,ot100059,1
8,ot100065,2
9,ot100087,5


## 2. 얼마나 많은 사람에게 주었는가

### 2.1.1 일방적으로 준경우 얼마나 많은 사람에게 주었는가?(from_how_many_one,to_how_many_one)

In [418]:
except_exchange.groupby(['trade_week','trade_day','trade_time','source_acc_id','target_acc_id'],as_index=False).count().groupby(['source_acc_id','target_acc_id'],as_index=False).count().iloc[:,:3]

,source_acc_id,target_acc_id,trade_week
0,ot100002,tr45819,1
1,ot100006,tr17846,7
2,ot100007,tr95350,1
3,ot10001,te28400,1
4,ot100013,te10428,1
5,ot100013,te15185,1
6,ot100013,te17579,1
7,ot100013,te22457,2
8,ot100013,te22926,1
9,ot100013,te23119,1


In [419]:
#source id별로 몇명에게 얼마나 나누어 주었는가

to_whom_one=except_exchange.groupby(['trade_week','trade_day','trade_time','source_acc_id','target_acc_id'],as_index=False).count().groupby(['source_acc_id','target_acc_id'],as_index=False).count().iloc[:,:3]
to_whom_one.columns=['source','target','cnt']
to_whom_one.head(30)

,source,target,cnt
0,ot100002,tr45819,1
1,ot100006,tr17846,7
2,ot100007,tr95350,1
3,ot10001,te28400,1
4,ot100013,te10428,1
5,ot100013,te15185,1
6,ot100013,te17579,1
7,ot100013,te22457,2
8,ot100013,te22926,1
9,ot100013,te23119,1


In [420]:
#몇번인지 파악
to_how_many_one=to_whom_one.groupby('source',as_index=False).count().iloc[:,:2]
to_how_many_one.columns=['source','to_how_many_cnt']
to_how_many_one.head()

,source,to_how_many_cnt
0,ot100002,1
1,ot100006,1
2,ot100007,1
3,ot10001,1
4,ot100013,50


In [421]:
#target id별로로 몇명한테 받았는가
from_whom_one=except_exchange.groupby(['trade_week','trade_day','trade_time','target_acc_id','source_acc_id'],as_index=False).count().groupby(['target_acc_id','source_acc_id'],as_index=False).count().iloc[:,:3]
from_whom_one.columns=['target','source','cnt']
from_whom_one.head(30)

,target,source,cnt
0,ot100013,te10428,1
1,ot100013,te11649,3
2,ot100013,te19218,1
3,ot100013,te22457,1
4,ot100013,te31338,3
5,ot100013,te31480,2
6,ot100013,te32172,2
7,ot100013,te36025,1
8,ot100013,te36995,4
9,ot100013,te3890,1


In [422]:
from_how_many_one=from_whom_one.groupby('target',as_index=False).count().iloc[:,:2]
from_how_many_one.columns=['target','from_how_many_cnt']
from_how_many_one.head()

,target,from_how_many_cnt
0,ot100013,47
1,ot100019,3
2,ot100036,1
3,ot100037,730
4,ot100039,15


### 2.1.2 일방향으로 준 경우 지니 불순도(to_gini_one,from_gini_one)


In [423]:
to_cnt_sum_one=to_whom_one.groupby('source',as_index=False)['cnt'].sum()
to_cnt_sum_one.columns=['source','cnt_sum']

to_gini_one=pd.merge(to_whom_one,to_cnt_sum_one)
del to_cnt_sum_one

In [424]:
#준 경우
to_gini_one['cnt']=to_gini_one['cnt']**2
to_gini_one['cnt_sum']=to_gini_one['cnt_sum']**2
to_gini_one['gini_per_class']=to_gini_one['cnt']/to_gini_one['cnt_sum']
to_gini_one=to_gini_one.groupby('source',as_index=False)['gini_per_class'].sum()
to_gini_one.columns=['source','give_gini']
to_gini_one.give_gini=1-to_gini_one.give_gini
to_gini_one.head(10)

,source,give_gini
0,ot100002,0.000000
1,ot100006,0.000000
2,ot100007,0.000000
3,ot10001,0.000000
4,ot100013,0.928342
5,ot100014,0.000000
6,ot100015,0.000000
7,ot100016,0.000000
8,ot100017,0.000000
9,ot100019,0.750000


In [427]:
sum(to_gini_one.give_gini>1)

0

In [428]:
from_cnt_sum_one=from_whom_one.groupby('target',as_index=False)['cnt'].sum()
from_cnt_sum_one.columns=['target','cnt_sum']

from_gini_one=pd.merge(from_whom_one,from_cnt_sum_one)
del from_cnt_sum_one

In [429]:
#받은 경우
from_gini_one['cnt']=from_gini_one['cnt']**2
from_gini_one['cnt_sum']=from_gini_one['cnt_sum']**2
from_gini_one['gini_per_class']=from_gini_one['cnt']/from_gini_one['cnt_sum']
from_gini_one=from_gini_one.groupby('target',as_index=False)['gini_per_class'].sum()
from_gini_one.columns=['target','rec_gini']
from_gini_one.rec_gini=1-from_gini_one.rec_gini
from_gini_one.head(10)

,target,rec_gini
0,ot100013,0.969955
1,ot100019,0.625000
2,ot100036,0.000000
3,ot100037,0.998513
4,ot100039,0.913223
5,ot10004,0.816327
6,ot100045,0.950255
7,ot100059,0.864198
8,ot100060,0.000000
9,ot100061,0.500000


In [432]:
sum(from_gini_one.rec_gini>1)

0

### 2.2.1. 교환한 경우 얼마나 많은 사람과 했는가?(with_how_many)

In [433]:
#source id별로 몇명에게 얼마나 나누어 주었는가
exchange_with=exchanges.groupby(['trade_week','trade_day','trade_time','target_acc_id','source_acc_id'],as_index=False).count().groupby(['source_acc_id','target_acc_id'],as_index=False).count().iloc[:,:3]
exchange_with.columns=['source','target','cnt']
exchange_with.head(30)

,source,target,cnt
0,ot100007,te26473,1
1,ot100007,te3519,1
2,ot100007,tr65912,1
3,ot100009,tr64082,4
4,ot100013,te11696,1
5,ot100013,te1382,1
6,ot100013,te22509,2
7,ot100013,te27353,1
8,ot100013,te27635,1
9,ot100013,te29709,1


In [434]:
#몇번인지 파악
with_how_many=exchange_with.groupby('source',as_index=False).count().iloc[:,:2]
with_how_many.columns=['source','with_how_many_cnt']
with_how_many.head()

,source,with_how_many_cnt
0,ot100007,3
1,ot100009,1
2,ot100013,45
3,ot100020,1
4,ot10003,2


### 2.2.2 교환한 경우 불순도(with_gini)

In [435]:
with_cnt_sum=exchange_with.groupby('source',as_index=False)['cnt'].sum()
with_cnt_sum.columns=['source','cnt_sum']

with_gini=pd.merge(exchange_with,with_cnt_sum)
del with_cnt_sum

In [436]:
#받은 경우
with_gini['cnt']=with_gini['cnt']**2
with_gini['cnt_sum']=with_gini['cnt_sum']**2
with_gini['gini_per_class']=with_gini['cnt']/with_gini['cnt_sum']
with_gini=with_gini.groupby('source',as_index=False)['gini_per_class'].sum()
with_gini.columns=['source','exc_gini']
with_gini.exc_gini=1-with_gini.exc_gini
with_gini.head(10)

,source,exc_gini
0,ot100007,0.666667
1,ot100009,0.000000
2,ot100013,0.882183
3,ot100020,0.000000
4,ot10003,0.375000
5,ot100039,0.000000
6,ot100045,0.750000
7,ot100059,0.000000
8,ot100065,0.500000
9,ot100087,0.560000


## 3. 거래한 category의 양 (여기선 exchange도 두개가 나와야 함)

### 3.1.1. 준 경우 category별  sum(give_category_sum_one)

In [437]:
give_category_sum_one=except_exchange.groupby(['source_acc_id','item_type'],as_index=False).sum()
give_category_sum_one.head()

,source_acc_id,item_type,trade_week,trade_day,item_amount
0,ot100002,grocery,1,3,8.0
1,ot100002,money,1,3,173962.0
2,ot100006,grocery,13,36,274.0
3,ot100006,money,13,36,1619226.0
4,ot100007,money,2,1,50000000.0


In [438]:
give_category_sum_one=give_category_sum_one.pivot('source_acc_id','item_type','item_amount')

In [439]:
give_category_sum_one=give_category_sum_one.fillna(0)

In [440]:
give_category_sum_one['source']=give_category_sum_one.index
give_category_sum_one.columns=['give_acc_sum','give_costume_sum','give_gem_sum','give_grocery_sum','give_money_sum','give_weapon_sum','source']
give_category_sum_one.head()

,give_acc_sum,give_costume_sum,give_gem_sum,give_grocery_sum,give_money_sum,give_weapon_sum,source
source_acc_id,,,,,,,
ot100002,0.0,0.0,0.0,8.0,1.739620e+05,0.0,ot100002
ot100006,0.0,0.0,0.0,274.0,1.619226e+06,0.0,ot100006
ot100007,0.0,0.0,0.0,0.0,5.000000e+07,0.0,ot100007
ot10001,0.0,0.0,0.0,0.0,3.000000e+06,0.0,ot10001
ot100013,0.0,1.0,0.0,1102.0,2.588653e+10,0.0,ot100013


### 3.1.2 준 경우 category별 회수 비율(give_category_ratio_one)

In [441]:
give_category_cnt_one=except_exchange.groupby(['source_acc_id','item_type'],as_index=False).count()
give_category_cnt_one.head()

,source_acc_id,item_type,trade_week,trade_day,trade_time,target_acc_id,item_amount
0,ot100002,grocery,1,1,1,1,1
1,ot100002,money,1,1,1,1,1
2,ot100006,grocery,7,7,7,7,7
3,ot100006,money,7,7,7,7,7
4,ot100007,money,1,1,1,1,1


In [442]:
give_category_cnt_one=give_category_cnt_one.pivot('source_acc_id','item_type','item_amount')
give_category_cnt_one=give_category_cnt_one.fillna(0)
give_category_cnt_one['cnt_sum']=give_category_cnt_one.sum(1)
give_category_cnt_one['source']=give_category_cnt_one.index

In [443]:
give_category_ratio_one=give_category_cnt_one.copy()
for i in give_category_ratio_one.columns[:-2]:
    give_category_ratio_one[i]/=give_category_ratio_one['cnt_sum']
del give_category_ratio_one['cnt_sum']
give_category_ratio_one.columns=['give_acc_ratio','give_costume_ratio','give_gem_ratio','give_grocery_ratio','give_money_ratio','give_weapon_ratio','source']
give_category_ratio_one.head()

,give_acc_ratio,give_costume_ratio,give_gem_ratio,give_grocery_ratio,give_money_ratio,give_weapon_ratio,source
source_acc_id,,,,,,,
ot100002,0.0,0.00000,0.0,0.500000,0.500000,0.0,ot100002
ot100006,0.0,0.00000,0.0,0.500000,0.500000,0.0,ot100006
ot100007,0.0,0.00000,0.0,0.000000,1.000000,0.0,ot100007
ot10001,0.0,0.00000,0.0,0.000000,1.000000,0.0,ot10001
ot100013,0.0,0.00885,0.0,0.159292,0.831858,0.0,ot100013


### 3.2.1 받은 경우 category별 sum(receive_category_sum_one)

In [444]:
receive_category_sum_one=except_exchange.groupby(['target_acc_id','item_type'],as_index=False).sum()
receive_category_sum_one.head()

,target_acc_id,item_type,trade_week,trade_day,item_amount
0,ot100013,grocery,4,1,5.000000e+00
1,ot100013,money,447,194,8.400910e+09
2,ot100019,money,9,22,1.327400e+08
3,ot100036,grocery,7,5,2.400000e+02
4,ot100037,grocery,7124,8736,3.028800e+04


In [445]:
receive_category_sum_one=receive_category_sum_one.pivot('target_acc_id','item_type','item_amount')

In [446]:
receive_category_sum_one=receive_category_sum_one.fillna(0)

In [447]:
receive_category_sum_one['target']=receive_category_sum_one.index
receive_category_sum_one.columns=['rec_acc_sum','rec_costume_sum','rec_gem_sum','rec_grocery_sum','rec_money_sum','rec_weapon_sum','target']
receive_category_sum_one.head()

,rec_acc_sum,rec_costume_sum,rec_gem_sum,rec_grocery_sum,rec_money_sum,rec_weapon_sum,target
target_acc_id,,,,,,,
ot100013,0.0,0.0,0.0,5.0,8.400910e+09,0.0,ot100013
ot100019,0.0,0.0,0.0,0.0,1.327400e+08,0.0,ot100019
ot100036,0.0,0.0,0.0,240.0,0.000000e+00,0.0,ot100036
ot100037,0.0,0.0,0.0,30288.0,1.390412e+08,0.0,ot100037
ot100039,0.0,0.0,0.0,0.0,8.604100e+09,0.0,ot100039


### 3.2.2 받은 경우 category 별 ratio(receive_category_ratio_one)

In [448]:
receive_category_cnt_one=except_exchange.groupby(['target_acc_id','item_type'],as_index=False).count()
receive_category_cnt_one.head()

,target_acc_id,item_type,trade_week,trade_day,trade_time,source_acc_id,item_amount
0,ot100013,grocery,1,1,1,1,1
1,ot100013,money,83,83,83,83,83
2,ot100019,money,4,4,4,4,4
3,ot100036,grocery,1,1,1,1,1
4,ot100037,grocery,1249,1249,1249,1249,1249


In [449]:
receive_category_cnt_one=receive_category_cnt_one.pivot('target_acc_id','item_type','item_amount')
receive_category_cnt_one=receive_category_cnt_one.fillna(0)
receive_category_cnt_one['cnt_sum']=receive_category_cnt_one.sum(1)
receive_category_cnt_one['target']=receive_category_cnt_one.index

In [450]:
receive_category_ratio_one=receive_category_cnt_one.copy()
for i in receive_category_ratio_one.columns[:-2]:
    receive_category_ratio_one[i]/=receive_category_ratio_one['cnt_sum']
del receive_category_ratio_one['cnt_sum']
receive_category_ratio_one.columns=['rec_acc_ratio','rec_costume_ratio','rec_gem_ratio','rec_grocery_ratio','rec_money_ratio','rec_weapon_ratio','target']
receive_category_ratio_one.head()

,rec_acc_ratio,rec_costume_ratio,rec_gem_ratio,rec_grocery_ratio,rec_money_ratio,rec_weapon_ratio,target
target_acc_id,,,,,,,
ot100013,0.0,0.0,0.0,0.011905,0.988095,0.0,ot100013
ot100019,0.0,0.0,0.0,0.000000,1.000000,0.0,ot100019
ot100036,0.0,0.0,0.0,1.000000,0.000000,0.0,ot100036
ot100037,0.0,0.0,0.0,0.500200,0.499800,0.0,ot100037
ot100039,0.0,0.0,0.0,0.000000,1.000000,0.0,ot100039


### 3.3.1 exchange의 경우 category 별 sum(give_categoty_sum_exc,receive_category_sum_exc)

In [451]:
give_category_sum_exc=exchanges.groupby(['source_acc_id','item_type'],as_index=False).sum()
give_category_sum_exc.head()

,source_acc_id,item_type,trade_week,trade_day,item_amount
0,ot100007,grocery,6,3,1.040000e+02
1,ot100009,grocery,31,17,9.460000e+02
2,ot100009,money,31,17,4.000834e+06
3,ot100013,grocery,344,212,1.618000e+03
4,ot100013,money,132,120,8.500000e+09


In [452]:
give_category_sum_exc=give_category_sum_exc.pivot('source_acc_id','item_type','item_amount')

In [453]:
give_category_sum_exc=give_category_sum_exc.fillna(0)

In [454]:
give_category_sum_exc['source']=give_category_sum_exc.index
give_category_sum_exc.columns=['give_exc_acc_sum_','give_exc_costume_sum_','give_exc_gem_sum','give_exc_grocery_sum','give_exc_money_sum','give_exc_weapon_sum','source']
give_category_sum_exc.head()

,give_exc_acc_sum_,give_exc_costume_sum_,give_exc_gem_sum,give_exc_grocery_sum,give_exc_money_sum,give_exc_weapon_sum,source
source_acc_id,,,,,,,
ot100007,0.0,0.0,0.0,104.0,0.000000e+00,0.0,ot100007
ot100009,0.0,0.0,0.0,946.0,4.000834e+06,0.0,ot100009
ot100013,0.0,0.0,0.0,1618.0,8.500000e+09,0.0,ot100013
ot100020,0.0,0.0,0.0,250.0,9.993540e+05,0.0,ot100020
ot10003,0.0,0.0,0.0,1010.0,4.007391e+06,0.0,ot10003


In [455]:
receive_category_sum_exc=exchanges.groupby(['target_acc_id','item_type'],as_index=False).sum()
receive_category_sum_exc.head()

,target_acc_id,item_type,trade_week,trade_day,item_amount
0,ot100007,money,6,3,8.650000e+07
1,ot100009,grocery,31,17,2.000000e+01
2,ot100013,gem,6,2,1.000000e+00
3,ot100013,grocery,137,125,3.520000e+02
4,ot100013,money,340,207,1.743115e+10


In [456]:
receive_category_sum_exc=receive_category_sum_exc.pivot('target_acc_id','item_type','item_amount')

In [457]:
receive_category_sum_exc=receive_category_sum_exc.fillna(0)

In [458]:
receive_category_sum_exc['target']=receive_category_sum_exc.index
receive_category_sum_exc.columns=['rec_exc_acc_sum','rec_exc_costume_sum','rec_exc_gem_sum','rec_exc_grocery_sum','rec_exc_money_sum','rec_exc_weapon_sum','target']
receive_category_sum_exc.head()

,rec_exc_acc_sum,rec_exc_costume_sum,rec_exc_gem_sum,rec_exc_grocery_sum,rec_exc_money_sum,rec_exc_weapon_sum,target
target_acc_id,,,,,,,
ot100007,0.0,0.0,0.0,0.0,8.650000e+07,0.0,ot100007
ot100009,0.0,0.0,0.0,20.0,0.000000e+00,0.0,ot100009
ot100013,0.0,0.0,1.0,352.0,1.743115e+10,0.0,ot100013
ot100020,0.0,0.0,0.0,5.0,0.000000e+00,0.0,ot100020
ot10003,0.0,0.0,0.0,20.0,0.000000e+00,0.0,ot10003


### 3.3.2 교환한 경우 category 별 비율(give_category_ratio_exc,receive_category_ratio_exc)

In [459]:
give_category_cnt_exc=exchanges.groupby(['source_acc_id','item_type'],as_index=False).count()
give_category_cnt_exc.head()

,source_acc_id,item_type,trade_week,trade_day,trade_time,target_acc_id,item_amount
0,ot100007,grocery,3,3,3,3,3
1,ot100009,grocery,4,4,4,4,4
2,ot100009,money,4,4,4,4,4
3,ot100013,grocery,59,59,59,59,59
4,ot100013,money,22,22,22,22,22


In [460]:
give_category_cnt_exc=give_category_cnt_exc.pivot('source_acc_id','item_type','item_amount')
give_category_cnt_exc=give_category_cnt_exc.fillna(0)
give_category_cnt_exc['cnt_sum']=give_category_cnt_exc.sum(1)
give_category_cnt_exc['source']=give_category_cnt_exc.index

In [461]:
give_category_ratio_exc=give_category_cnt_exc.copy()
for i in give_category_ratio_exc.columns[:-2]:
    give_category_ratio_exc[i]/=give_category_ratio_exc['cnt_sum']
del give_category_ratio_exc['cnt_sum']
give_category_ratio_exc.columns=['give_exc_acc_ratio','give_exc_costume_ratio','give_exc_gem_ratio','give_exc_grocery_ratio','give_exc_money_ratio','give_exc_weapon_ratio','source']

give_category_ratio_exc.head()

,give_exc_acc_ratio,give_exc_costume_ratio,give_exc_gem_ratio,give_exc_grocery_ratio,give_exc_money_ratio,give_exc_weapon_ratio,source
source_acc_id,,,,,,,
ot100007,0.0,0.0,0.0,1.000000,0.000000,0.0,ot100007
ot100009,0.0,0.0,0.0,0.500000,0.500000,0.0,ot100009
ot100013,0.0,0.0,0.0,0.728395,0.271605,0.0,ot100013
ot100020,0.0,0.0,0.0,0.500000,0.500000,0.0,ot100020
ot10003,0.0,0.0,0.0,0.500000,0.500000,0.0,ot10003


In [462]:
receive_category_cnt_exc=exchanges.groupby(['target_acc_id','item_type'],as_index=False).count()
receive_category_cnt_exc.head()

,target_acc_id,item_type,trade_week,trade_day,trade_time,source_acc_id,item_amount
0,ot100007,money,3,3,3,3,3
1,ot100009,grocery,4,4,4,4,4
2,ot100013,gem,1,1,1,1,1
3,ot100013,grocery,23,23,23,23,23
4,ot100013,money,58,58,58,58,58


In [463]:
receive_category_cnt_exc=receive_category_cnt_exc.pivot('target_acc_id','item_type','item_amount')
receive_category_cnt_exc=receive_category_cnt_exc.fillna(0)
receive_category_cnt_exc['cnt_sum']=receive_category_cnt_exc.sum(1)
receive_category_cnt_exc['target']=receive_category_cnt_exc.index

In [464]:
receive_category_ratio_exc=receive_category_cnt_exc.copy()
for i in receive_category_ratio_exc.columns[:-2]:
    receive_category_ratio_exc[i]/=receive_category_ratio_exc['cnt_sum']
del receive_category_ratio_exc['cnt_sum']
receive_category_ratio_exc.columns=['rec_exc_acc_ratio','rec_exc_costume_ratio','rec_exc_gem_ratio','rec_exc_grocery_ratio','rec_exc_money_ratio','rec_exc_weapon_ratio','source']
receive_category_ratio_exc.head()

,rec_exc_acc_ratio,rec_exc_costume_ratio,rec_exc_gem_ratio,rec_exc_grocery_ratio,rec_exc_money_ratio,rec_exc_weapon_ratio,source
target_acc_id,,,,,,,
ot100007,0.0,0.0,0.000000,0.000000,1.000000,0.0,ot100007
ot100009,0.0,0.0,0.000000,1.000000,0.000000,0.0,ot100009
ot100013,0.0,0.0,0.012195,0.280488,0.707317,0.0,ot100013
ot100020,0.0,0.0,0.000000,1.000000,0.000000,0.0,ot100020
ot10003,0.0,0.0,0.000000,1.000000,0.000000,0.0,ot10003


## 4. 어느 주차에 가장 많이 trade 했는가

### 4.1 준 경우 어느 주차에 가장 많이 주었는가, 마지막으로 언제 주었는가?(give_last_week,give_most_weeks)

In [465]:
give_by_week=except_exchange.groupby(['source_acc_id','trade_week'],as_index=False).count()
give_which_week=give_by_week.pivot('source_acc_id','trade_week','trade_day')
give_which_week=give_which_week.fillna(0)
give_which_week['source']=give_which_week.index
give_which_week.head()

trade_week,1,2,3,4,5,6,7,8,source
source_acc_id,,,,,,,,,
ot100002,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ot100002
ot100006,4.0,8.0,2.0,0.0,0.0,0.0,0.0,0.0,ot100006
ot100007,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ot100007
ot10001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ot10001
ot100013,2.0,18.0,9.0,21.0,5.0,31.0,13.0,14.0,ot100013


In [466]:
#가장 마지막 준 주차
give_last_week=give_by_week.groupby('source_acc_id',as_index=False)['trade_week'].max()
give_last_week.columns=['source','give_last']
give_last_week.head(7)

,source,give_last
0,ot100002,1
1,ot100006,3
2,ot100007,2
3,ot10001,1
4,ot100013,8
5,ot100014,2
6,ot100015,1


In [467]:
#가장 많이준 1,2,3위 주차
give_most_weeks=give_by_week.sort_values(['source_acc_id','trade_day'],ascending=False)
#어쩔 수 없이 first를 씀
give_most_weeks['trade_day']=give_most_weeks.groupby('source_acc_id')['trade_day'].rank(ascending=False,method='first').astype(int)
give_most_weeks.head()

,source_acc_id,trade_week,trade_day,trade_time,target_acc_id,item_type,item_amount
464148,tr99998,2,1,9,9,9,9
464151,tr99998,8,2,7,7,7,7
464147,tr99998,1,3,4,4,4,4
464149,tr99998,4,4,3,3,3,3
464150,tr99998,7,5,2,2,2,2


In [468]:
give_most_weeks=give_most_weeks.pivot('source_acc_id','trade_day','trade_week').iloc[:,:3]
give_most_weeks.columns=['give_first','give_second','give_third']
give_most_weeks['source']=give_most_weeks.index

In [469]:
give_most_weeks.head()

,give_first,give_second,give_third,source
source_acc_id,,,,
ot100002,1.0,NaN,NaN,ot100002
ot100006,2.0,1.0,3.0,ot100006
ot100007,2.0,NaN,NaN,ot100007
ot10001,1.0,NaN,NaN,ot10001
ot100013,6.0,4.0,2.0,ot100013


In [470]:
#실제 값과 비교, 맞는 것을 확인
give_by_week.head()

,source_acc_id,trade_week,trade_day,trade_time,target_acc_id,item_type,item_amount
0,ot100002,1,2,2,2,2,2
1,ot100006,1,4,4,4,4,4
2,ot100006,2,8,8,8,8,8
3,ot100006,3,2,2,2,2,2
4,ot100007,2,1,1,1,1,1


### 4.2 받은 경우 어느 주차에 가장 많이 받았는가, 마지막으로 언제 받았는가?(receive_last_week,receive_most_weeks)

In [471]:
receive_by_week=except_exchange.groupby(['target_acc_id','trade_week'],as_index=False).count()
receive_which_week=receive_by_week.pivot('target_acc_id','trade_week','trade_day')
receive_which_week=receive_which_week.fillna(0)
receive_which_week['target']=receive_which_week.index
receive_which_week.head()

trade_week,1,2,3,4,5,6,7,8,target
target_acc_id,,,,,,,,,
ot100013,5.0,9.0,6.0,10.0,2.0,22.0,12.0,18.0,ot100013
ot100019,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,ot100019
ot100036,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,ot100036
ot100037,0.0,0.0,254.0,0.0,0.0,2214.0,29.0,0.0,ot100037
ot100039,2.0,4.0,4.0,0.0,3.0,5.0,1.0,3.0,ot100039


In [472]:
#가장 마지막 준 주차
receive_last_week=receive_by_week.groupby('target_acc_id',as_index=False)['trade_week'].max()
receive_last_week.columns=['target','rec_last']
receive_last_week.head(7)

,target,rec_last
0,ot100013,8
1,ot100019,3
2,ot100036,7
3,ot100037,7
4,ot100039,8
5,ot10004,8
6,ot100045,8


In [473]:
#가장 많이 받은 1,2,3위 주차
receive_most_weeks=receive_by_week.sort_values(['target_acc_id','trade_day'],ascending=False)
#어쩔 수 없이 first를 씀
receive_most_weeks['trade_day']=receive_most_weeks.groupby('target_acc_id')['trade_day'].rank(ascending=False,method='first').astype(int)
receive_most_weeks.head()

,target_acc_id,trade_week,trade_day,trade_time,source_acc_id,item_type,item_amount
192484,tr99998,8,1,3,3,3,3
192480,tr99998,2,2,2,2,2,2
192481,tr99998,3,3,2,2,2,2
192482,tr99998,4,4,2,2,2,2
192483,tr99998,7,5,2,2,2,2


In [474]:
receive_most_weeks=receive_most_weeks.pivot('target_acc_id','trade_day','trade_week').iloc[:,:3]
receive_most_weeks.columns=['rec_first','rec_second','rec_third']
receive_most_weeks['source']=receive_most_weeks.index

In [475]:
receive_most_weeks.head()

,rec_first,rec_second,rec_third,source
target_acc_id,,,,
ot100013,6.0,8.0,7.0,ot100013
ot100019,3.0,1.0,2.0,ot100019
ot100036,7.0,NaN,NaN,ot100036
ot100037,6.0,3.0,7.0,ot100037
ot100039,6.0,2.0,3.0,ot100039


In [476]:
#실제 값과 비교, 맞는 것을 확인
receive_by_week.head(10)

,target_acc_id,trade_week,trade_day,trade_time,source_acc_id,item_type,item_amount
0,ot100013,1,5,5,5,5,5
1,ot100013,2,9,9,9,9,9
2,ot100013,3,6,6,6,6,6
3,ot100013,4,10,10,10,10,10
4,ot100013,5,2,2,2,2,2
5,ot100013,6,22,22,22,22,22
6,ot100013,7,12,12,12,12,12
7,ot100013,8,18,18,18,18,18
8,ot100019,1,1,1,1,1,1
9,ot100019,2,1,1,1,1,1


### 4.3 교환 경우 어느 주차에 가장 많이 했는가, 마지막으로 언제 했는가?(exc_last_week,exc_most_weeks)

In [477]:
exc_by_week=exchanges.groupby(['source_acc_id','trade_week'],as_index=False).count()
exc_which_week=exc_by_week.pivot('source_acc_id','trade_week','trade_day')
exc_which_week=exc_which_week.fillna(0)
exc_which_week['source']=exc_which_week.index
exc_which_week.head()

trade_week,1,2,3,4,5,6,7,8,source
source_acc_id,,,,,,,,,
ot100007,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,ot100007
ot100009,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,ot100009
ot100013,0.0,5.0,1.0,7.0,1.0,44.0,18.0,5.0,ot100013
ot100020,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,ot100020
ot10003,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,ot10003


In [478]:
#가장 마지막 준 주차
exc_last_week=exc_by_week.groupby('source_acc_id',as_index=False)['trade_week'].max()
exc_last_week.columns=['source','exc_last']
exc_last_week.head(7)

,source,exc_last
0,ot100007,2
1,ot100009,8
2,ot100013,8
3,ot100020,7
4,ot10003,8
5,ot100039,7
6,ot100045,8


In [479]:
#가장 많이준 1,2,3위 주차
exc_most_weeks=exc_by_week.sort_values(['source_acc_id','trade_day'],ascending=False)
#어쩔 수 없이 first를 씀
exc_most_weeks['trade_day']=exc_most_weeks.groupby('source_acc_id')['trade_day'].rank(ascending=False,method='first').astype(int)
exc_most_weeks.head()

,source_acc_id,trade_week,trade_day,trade_time,target_acc_id,item_type,item_amount
96687,tr99998,3,1,1,1,1,1
96686,tr99994,7,1,4,4,4,4
96685,tr99994,6,2,2,2,2,2
96682,tr99994,1,3,1,1,1,1
96683,tr99994,2,4,1,1,1,1


In [480]:
exc_most_weeks=exc_most_weeks.pivot('source_acc_id','trade_day','trade_week').iloc[:,:3]
exc_most_weeks.columns=['exc_first','exc_second','exc_third']
exc_most_weeks['source']=exc_most_weeks.index

In [481]:
exc_most_weeks.head()

,exc_first,exc_second,exc_third,source
source_acc_id,,,,
ot100007,2.0,NaN,NaN,ot100007
ot100009,8.0,7.0,NaN,ot100009
ot100013,6.0,7.0,4.0,ot100013
ot100020,7.0,NaN,NaN,ot100020
ot10003,7.0,8.0,NaN,ot10003


In [482]:
#실제 값과 비교, 맞는 것을 확인
exc_by_week.head(10)

,source_acc_id,trade_week,trade_day,trade_time,target_acc_id,item_type,item_amount
0,ot100007,2,3,3,3,3,3
1,ot100009,7,2,2,2,2,2
2,ot100009,8,6,6,6,6,6
3,ot100013,2,5,5,5,5,5
4,ot100013,3,1,1,1,1,1
5,ot100013,4,7,7,7,7,7
6,ot100013,5,1,1,1,1,1
7,ot100013,6,44,44,44,44,44
8,ot100013,7,18,18,18,18,18
9,ot100013,8,5,5,5,5,5


# Merging tables

## 1. Merge의 기준이 될 데이터

In [483]:
training_ids=pd.read_csv('lite_data/train_label_lite.csv')
training_ids.head()

,new_id,label
0,tr0,month
1,tr1,week
2,tr2,retained
3,tr3,week
4,tr4,retained


In [484]:
len(training_ids)

100000

In [485]:
test_ids=training_ids.copy()
test_ids['new_id']=test_ids['new_id'].str.replace('tr','te').tolist()
test_ids=test_ids.iloc[:40000,:]
test_ids.head()
#!!!!! label값은 의미없는 값임

,new_id,label
0,te0,month
1,te1,week
2,te2,retained
3,te3,week
4,te4,retained


In [486]:
len(test_ids)

40000

In [487]:
type(source_cnt_one.iloc[0,0])==str

True

In [488]:
#data들의 list
datas=[source_cnt_one,target_cnt_one,exchange_cnt,
       from_how_many_one,to_how_many_one,with_how_many,
       to_gini_one,from_gini_one,with_gini,
       give_category_sum_one,receive_category_sum_one,give_category_sum_exc,receive_category_sum_exc,
       give_category_ratio_one,receive_category_ratio_one,give_category_ratio_exc,receive_category_ratio_exc,
       give_last_week,receive_last_week,exc_last_week,
       give_most_weeks,receive_most_weeks,exc_most_weeks]

In [489]:
#어떤 column에 id가 있는지 저장
id_col=[]
for i in range(len(datas)):
    print(len(datas[i].columns))
    for j in range(len(datas[i].columns)):
        if type(datas[i].iloc[0,j])==str:
            id_col.append(datas[i].columns[j])
            continue
print(id_col)

2
2
2
2
2
2
2
2
2
7
7
7
7
7
7
7
7
2
2
2
4
4
4
['source_id', 'traget_id', 'id', 'target', 'source', 'source', 'source', 'target', 'source', 'source', 'target', 'source', 'target', 'source', 'target', 'source', 'source', 'source', 'target', 'source', 'source', 'source', 'source']


In [490]:
aa=pd.merge(training_ids,datas[0],left_on='new_id',right_on='source_id',how='left')

### 5.2 training 데이터 merge, 저장

In [504]:
for i in range(len(datas)):
    if i ==0:
        training_trade=pd.merge(training_ids,datas[i],left_on='new_id',right_on=id_col[i],how='left')
        del training_trade[id_col[i]]
    else:
        training_trade=pd.merge(training_trade,datas[i],left_on='new_id',right_on=id_col[i],how='left')
        del training_trade[id_col[i]]

In [505]:
training_trade.head()

,new_id,label,source_cnt,target_cnt,exchange_cnt,from_how_many_cnt,to_how_many_cnt,with_how_many_cnt,give_gini,rec_gini,...,exc_last,give_first,give_second,give_third,rec_first,rec_second,rec_third,exc_first,exc_second,exc_third
0,tr0,month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tr1,week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tr2,retained,8.0,NaN,NaN,NaN,1.0,NaN,0.000000,NaN,...,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tr3,week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tr4,retained,18.0,14.0,14.0,11.0,8.0,14.0,0.808642,0.890625,...,7.0,1.0,2.0,5.0,5.0,3.0,1.0,2.0,3.0,1.0


In [506]:
training_trade.to_csv('Features/train_trade_with_label.csv',index=False)

In [507]:
del training_trade['label']
training_trade.to_csv('Features/train_trade.csv',index = False)

### 5.3 test 데이터 merge, 저장

In [508]:
for i in range(len(datas)):
    if i ==0:
        test_trade=pd.merge(test_ids,datas[i],left_on='new_id',right_on=id_col[i],how='left')
        del test_trade[id_col[i]]
    else:
        test_trade=pd.merge(test_trade,datas[i],left_on='new_id',right_on=id_col[i],how='left')
        del test_trade[id_col[i]]

In [509]:
test_trade.head()

,new_id,label,source_cnt,target_cnt,exchange_cnt,from_how_many_cnt,to_how_many_cnt,with_how_many_cnt,give_gini,rec_gini,...,exc_last,give_first,give_second,give_third,rec_first,rec_second,rec_third,exc_first,exc_second,exc_third
0,te0,month,32.0,NaN,NaN,17.0,17.0,NaN,0.919922,0.890533,...,NaN,5.0,1.0,3.0,3.0,2.0,1.0,NaN,NaN,NaN
1,te1,week,77.0,13.0,13.0,13.0,16.0,4.0,0.746500,0.661600,...,8.0,2.0,1.0,4.0,7.0,8.0,3.0,6.0,2.0,3.0
2,te2,retained,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,te3,week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,te4,retained,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [510]:
del test_trade['label']
test_trade.to_csv('Features/test_trade.csv',index = False)

In [499]:
from sklearn import tree

In [500]:
clf = tree.DecisionTreeClassifier(min_samples_split=300,min_samples_leaf=300,max_depth=15,splitter='best')
#for i in 

In [501]:
labels=pd.read_csv('FE and error_set/error_set_0.741.csv')

# 4. 추가 생성 변수 ver1.


In [6]:
train_trade=pd.read_csv('Features/train_trade.csv')
test_trade=pd.read_csv('Features/test_trade.csv')

# 추가 1. give+exchange, target+exchage

In [10]:
train_trade['source+exc']=train_trade.source_cnt.fillna(0)+train_trade.exchange_cnt.fillna(0)
train_trade['target+exc']=train_trade.target_cnt.fillna(0)+train_trade.exchange_cnt.fillna(0)

In [11]:
test_trade['source+exc']=test_trade.source_cnt.fillna(0)+test_trade.exchange_cnt.fillna(0)
test_trade['target+exc']=test_trade.target_cnt.fillna(0)+test_trade.exchange_cnt.fillna(0)

# 추가 2. 전체 거래 중 give 거래 비율, exchange 거래 비율

In [14]:
train_trade['exc_ratio']=train_trade.exchange_cnt.fillna(0)/(train_trade.source_cnt.fillna(0)+train_trade.target_cnt.fillna(0)+train_trade.exchange_cnt.fillna(0))
train_trade['one_way_ratio']=(train_trade.source_cnt.fillna(0)+train_trade.target_cnt.fillna(0))/(train_trade.source_cnt.fillna(0)+train_trade.target_cnt.fillna(0)+train_trade.exchange_cnt.fillna(0))

In [15]:
test_trade['exc_ratio']=test_trade.exchange_cnt.fillna(0)/(test_trade.source_cnt.fillna(0)+test_trade.target_cnt.fillna(0)+test_trade.exchange_cnt.fillna(0))
test_trade['one_way_ratio']=(test_trade.source_cnt.fillna(0)+test_trade.target_cnt.fillna(0))/(test_trade.source_cnt.fillna(0)+test_trade.target_cnt.fillna(0)+test_trade.exchange_cnt.fillna(0))

In [16]:
train_trade.head()

,new_id,source_cnt,target_cnt,exchange_cnt,from_how_many_cnt,to_how_many_cnt,with_how_many_cnt,give_gini,rec_gini,exc_gini,...,rec_first,rec_second,rec_third,exc_first,exc_second,exc_third,source+exc,target+exc,exc_ratio,one_way_ratio
0,tr0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
1,tr1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
2,tr2,8.0,NaN,NaN,NaN,1.0,NaN,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8.0,0.0,0.000000,1.000000
3,tr3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
4,tr4,18.0,14.0,14.0,11.0,8.0,14.0,0.808642,0.890625,0.928571,...,5.0,3.0,1.0,2.0,3.0,1.0,32.0,28.0,0.304348,0.695652


# 추가3. 주차별 거래회수 차이에 관한 변수

In [88]:
multiple_transactions_final=pd.read_csv('Features/trade_with_exchange_bool.csv')
multiple_transactions_final.head()

,trade_week,trade_day,trade_time,source_acc_id,target_acc_id,item_type,item_amount,is_exchange
0,1,1,00:00:00,ot51070,tr88265,grocery,2.0,0.0
1,1,1,00:00:00,ot51070,tr88265,money,285122.0,0.0
2,1,1,00:00:00,ot127363,te1423,money,2200000.0,0.0
3,1,1,00:00:02,tr39831,ot97116,grocery,1.0,0.0
4,1,1,00:00:02,tr39831,ot97116,money,285134.0,0.0


In [89]:
weekly_give=multiple_transactions_final[multiple_transactions_final.is_exchange==0].groupby(['trade_week','trade_day','trade_time','source_acc_id','target_acc_id'],as_index=False).count().groupby(['trade_week','source_acc_id'],as_index=False).count()
weekly_get=multiple_transactions_final[multiple_transactions_final.is_exchange==0].groupby(['trade_week','trade_day','trade_time','source_acc_id','target_acc_id'],as_index=False).count().groupby(['trade_week','target_acc_id'],as_index=False).count()
weekly_exc=multiple_transactions_final[multiple_transactions_final.is_exchange==1].groupby(['trade_week','trade_day','trade_time','source_acc_id','target_acc_id'],as_index=False).count().groupby(['trade_week','source_acc_id'],as_index=False).count()

In [90]:
weekly_give=weekly_give.sort_values('source_acc_id').iloc[:,:3]
weekly_get=weekly_get.sort_values('target_acc_id').iloc[:,:3]
weekly_exc=weekly_exc.sort_values('source_acc_id').iloc[:,:3]
weekly_exc.head()

,trade_week,source_acc_id,trade_day
7901,2,ot100007,3
76640,8,ot100009,3
61080,7,ot100009,1
17836,3,ot100013,1
29436,4,ot100013,7


In [91]:
weekly_give.columns=['trade_week','new_id','trade_count']
weekly_get.columns=['trade_week','new_id','trade_count']
weekly_exc.columns=['trade_week','new_id','trade_count']

In [94]:
weekly_trade_count=pd.merge(weekly_give,weekly_get,on=['trade_week','new_id'],how='outer')
weekly_trade_count=pd.merge(weekly_trade_count,weekly_exc,on=['trade_week','new_id'],how='outer')
weekly_trade_count=weekly_trade_count.fillna(0)
weekly_trade_count.head(30)

,trade_week,new_id,trade_count_x,trade_count_y,trade_count
0,1,ot100002,1.0,0.0,0.0
1,3,ot100006,1.0,0.0,0.0
2,2,ot100006,4.0,0.0,0.0
3,1,ot100006,2.0,0.0,0.0
4,2,ot100007,1.0,0.0,3.0
5,1,ot10001,1.0,0.0,0.0
6,5,ot100013,5.0,2.0,1.0
7,6,ot100013,31.0,22.0,44.0
8,4,ot100013,21.0,10.0,7.0
9,7,ot100013,13.0,12.0,18.0


In [95]:
weekly_trade_count['trade_cnt_sum']=weekly_trade_count.iloc[:,2:].sum(1)
weekly_trade_count.head()

,trade_week,new_id,trade_count_x,trade_count_y,trade_count,trade_cnt_sum
0,1,ot100002,1.0,0.0,0.0,1.0
1,3,ot100006,1.0,0.0,0.0,1.0
2,2,ot100006,4.0,0.0,0.0,4.0
3,1,ot100006,2.0,0.0,0.0,2.0
4,2,ot100007,1.0,0.0,3.0,4.0


In [96]:
weekly_trade_count=weekly_trade_count.loc[:,['trade_week','new_id','trade_cnt_sum']]

In [99]:
weekly_trade_count_pivot=weekly_trade_count.pivot('new_id','trade_week','trade_cnt_sum')
weekly_trade_count_pivot['new_id']=weekly_trade_count_pivot.index
weekly_trade_count_pivot=weekly_trade_count_pivot.fillna(0)
weekly_trade_count_pivot.head()

trade_week,1,2,3,4,5,6,7,8,new_id
new_id,,,,,,,,,
ot100002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ot100002
ot100006,2.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,ot100006
ot100007,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,ot100007
ot100009,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,ot100009
ot10001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ot10001


In [101]:
last_trade_all_week=pd.DataFrame(weekly_trade_count.groupby('new_id')['trade_week'].max(),weekly_trade_count.groupby('new_id')['trade_week'].max().index)
last_trade_all_week['new_id']=last_trade_all_week.index
last_trade_all_week.index=range(len(last_trade_all_week))
last_trade_all_week.columns=['last_trade_all','new_id']

first_trade_all_week=pd.DataFrame(weekly_trade_count.groupby('new_id')['trade_week'].min(),weekly_trade_count.groupby('new_id')['trade_week'].min().index)
first_trade_all_week['new_id']=first_trade_all_week.index
first_trade_all_week.index=range(len(first_trade_all_week))
first_trade_all_week.columns=['first_trade_all','new_id']

first_trade_all_week.head()

,first_trade_all,new_id
0,1,ot100002
1,1,ot100006
2,2,ot100007
3,7,ot100009
4,1,ot10001


In [102]:
weekly_trade_count_pivot=pd.merge(weekly_trade_count_pivot,last_trade_all_week,on='new_id',how='left')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: FutureWarning: 'new_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [103]:
weekly_trade_count_pivot=pd.merge(weekly_trade_count_pivot,first_trade_all_week,on='new_id',how='left')

In [104]:
weekly_trade_count_pivot.head(10)

,1,2,3,4,5,6,7,8,new_id,last_trade_all,first_trade_all
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ot100002,1,1
1,2.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,ot100006,3,1
2,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,ot100007,2,2
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,ot100009,8,7
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ot10001,1,1
5,7.0,32.0,16.0,38.0,8.0,97.0,43.0,37.0,ot100013,8,1
6,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ot100014,2,1
7,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ot100015,1,1
8,5.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,ot100016,3,1
9,7.0,6.0,5.0,7.0,2.0,0.0,0.0,0.0,ot100017,5,1


In [105]:
weekly_trade_count_pivot=weekly_trade_count_pivot.fillna(0)
weekly_trade_count_pivot['last_week_trade_cnt']=weekly_trade_count_pivot.apply((lambda x:x.iloc[(x.iloc[9]-1)]),1)
weekly_trade_count_pivot['first_week_trade_cnt']=weekly_trade_count_pivot.apply((lambda x:x.iloc[(x.iloc[10]-1)]),1)
weekly_trade_count_pivot['weekly_max_trade_cnt']=weekly_trade_count_pivot.iloc[:,:8].max(1)
weekly_trade_count_pivot['weekly_trade_cnt_std']=weekly_trade_count_pivot.iloc[:,:8].std(1)
weekly_trade_count_pivot['weekly_trade_cycle']=weekly_trade_count_pivot.iloc[:,:8].apply((lambda x:x.autocorr()),1)


In [106]:
weekly_trade_count_pivot['trade_first-last']=weekly_trade_count_pivot.first_week_trade_cnt-weekly_trade_count_pivot.last_week_trade_cnt
weekly_trade_count_pivot['trade_last-max']=weekly_trade_count_pivot.last_week_trade_cnt-weekly_trade_count_pivot.weekly_max_trade_cnt

In [ ]:
last_diff_first, last_diff_max, std, cycle

In [107]:
weekly_trade_count_pivot.head()

,1,2,3,4,5,6,7,8,new_id,last_trade_all,first_trade_all,last_week_trade_cnt,first_week_trade_cnt,weekly_max_trade_cnt,weekly_trade_cnt_std,weekly_trade_cycle,trade_first-last,trade_last-max
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ot100002,1,1,1.0,1.0,1.0,0.353553,NaN,0.0,0.0
1,2.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,ot100006,3,1,1.0,2.0,4.0,1.457738,0.510527,1.0,-3.0
2,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,ot100007,2,2,4.0,4.0,4.0,1.414214,-0.166667,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,ot100009,8,7,3.0,1.0,3.0,1.069045,0.944444,-2.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ot10001,1,1,1.0,1.0,1.0,0.353553,NaN,0.0,0.0


In [109]:
train_trade=pd.merge(train_trade,weekly_trade_count_pivot.iloc[:,7:],on='new_id',how='left')
train_trade.head()

,new_id,source_cnt,target_cnt,exchange_cnt,from_how_many_cnt,to_how_many_cnt,with_how_many_cnt,give_gini,rec_gini,exc_gini,...,8,last_trade_all,first_trade_all,last_week_trade_cnt,first_week_trade_cnt,weekly_max_trade_cnt,weekly_trade_cnt_std,weekly_trade_cycle,trade_first-last,trade_last-max
0,tr0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tr1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tr2,8.0,NaN,NaN,NaN,1.0,NaN,0.000000,NaN,NaN,...,8.0,8.0,8.0,8.0,8.0,8.0,2.828427,NaN,0.0,0.0
3,tr3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tr4,18.0,14.0,14.0,11.0,8.0,14.0,0.808642,0.890625,0.928571,...,0.0,7.0,1.0,2.0,10.0,11.0,4.535574,0.487443,8.0,-9.0


In [110]:
test_trade=pd.merge(test_trade,weekly_trade_count_pivot.iloc[:,7:],on='new_id',how='left')
test_trade.head()

,new_id,source_cnt,target_cnt,exchange_cnt,from_how_many_cnt,to_how_many_cnt,with_how_many_cnt,give_gini,rec_gini,exc_gini,...,8,last_trade_all,first_trade_all,last_week_trade_cnt,first_week_trade_cnt,weekly_max_trade_cnt,weekly_trade_cnt_std,weekly_trade_cycle,trade_first-last,trade_last-max
0,te0,32.0,NaN,NaN,17.0,17.0,NaN,0.919922,0.890533,NaN,...,2.0,8.0,1.0,2.0,10.0,11.0,3.195980,0.618236,8.0,-9.0
1,te1,77.0,13.0,13.0,13.0,16.0,4.0,0.746500,0.661600,0.627219,...,19.0,8.0,1.0,19.0,17.0,23.0,4.140393,0.243385,-2.0,-4.0
2,te2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,te3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,te4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
train_trade.to_csv('Features/train_trade.csv',index=False)
test_trade.to_csv('Features/test_trade.csv',index=False)

# 관련 issue
## 1. NA값 처리

In [511]:
training_trade.columns

Index(['new_id', 'source_cnt', 'target_cnt', 'exchange_cnt',
       'from_how_many_cnt', 'to_how_many_cnt', 'with_how_many_cnt',
       'give_gini', 'rec_gini', 'exc_gini', 'give_acc_sum', 'give_costume_sum',
       'give_gem_sum', 'give_grocery_sum', 'give_money_sum', 'give_weapon_sum',
       'rec_acc_sum', 'rec_costume_sum', 'rec_gem_sum', 'rec_grocery_sum',
       'rec_money_sum', 'rec_weapon_sum', 'give_exc_acc_sum_',
       'give_exc_costume_sum_', 'give_exc_gem_sum', 'give_exc_grocery_sum',
       'give_exc_money_sum', 'give_exc_weapon_sum', 'rec_exc_acc_sum',
       'rec_exc_costume_sum', 'rec_exc_gem_sum', 'rec_exc_grocery_sum',
       'rec_exc_money_sum', 'rec_exc_weapon_sum', 'give_acc_ratio',
       'give_costume_ratio', 'give_gem_ratio', 'give_grocery_ratio',
       'give_money_ratio', 'give_weapon_ratio', 'rec_acc_ratio',
       'rec_costume_ratio', 'rec_gem_ratio', 'rec_grocery_ratio',
       'rec_money_ratio', 'rec_weapon_ratio', 'give_exc_acc_ratio',
       'give_exc_c

### Na값 padding principle
#### cnt,sum,ratio의 경우 0으로 패딩
#### gini의 경우 1로 패딩
#### last,first,second,third의 경우 고민을 해봐야함

In [512]:
for i in training_trade.columns:
    if 'cnt' in i:
        training_trade[i]=training_trade[i].fillna(0)
        test_trade[i]=test_trade[i].fillna(0)
    elif 'sum' in i:
        training_trade[i]=training_trade[i].fillna(0)
        test_trade[i]=test_trade[i].fillna(0)
    elif 'ratio' in i:
        training_trade[i]=training_trade[i].fillna(0)
        test_trade[i]=test_trade[i].fillna(0)
    elif 'gini' in i:
        training_trade[i]=training_trade[i].fillna(1)
        test_trade[i]=test_trade[i].fillna(1)
    elif 'last' in i:
        training_trade[i]=training_trade[i].fillna(20)
        test_trade[i]=test_trade[i].fillna(20)
    elif 'first' in i:
        training_trade[i]=training_trade[i].fillna(20)
        test_trade[i]=test_trade[i].fillna(20)
    elif 'second' in i:
        training_trade[i]=training_trade[i].fillna(20)
        test_trade[i]=test_trade[i].fillna(20)
    elif 'third' in i:
        training_trade[i]=training_trade[i].fillna(20)
        test_trade[i]=test_trade[i].fillna(20)


In [513]:
training_trade.head()

,new_id,source_cnt,target_cnt,exchange_cnt,from_how_many_cnt,to_how_many_cnt,with_how_many_cnt,give_gini,rec_gini,exc_gini,...,exc_last,give_first,give_second,give_third,rec_first,rec_second,rec_third,exc_first,exc_second,exc_third
0,tr0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
1,tr1,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2,tr2,8.0,0.0,0.0,0.0,1.0,0.0,0.000000,1.000000,1.000000,...,20.0,8.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
3,tr3,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
4,tr4,18.0,14.0,14.0,11.0,8.0,14.0,0.808642,0.890625,0.928571,...,7.0,1.0,2.0,5.0,5.0,3.0,1.0,2.0,3.0,1.0


In [514]:
test_trade.head()

,new_id,source_cnt,target_cnt,exchange_cnt,from_how_many_cnt,to_how_many_cnt,with_how_many_cnt,give_gini,rec_gini,exc_gini,...,exc_last,give_first,give_second,give_third,rec_first,rec_second,rec_third,exc_first,exc_second,exc_third
0,te0,32.0,0.0,0.0,17.0,17.0,0.0,0.919922,0.890533,1.000000,...,20.0,5.0,1.0,3.0,3.0,2.0,1.0,20.0,20.0,20.0
1,te1,77.0,13.0,13.0,13.0,16.0,4.0,0.746500,0.661600,0.627219,...,8.0,2.0,1.0,4.0,7.0,8.0,3.0,6.0,2.0,3.0
2,te2,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
3,te3,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
4,te4,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0


In [5]:
training_trade.to_csv('Trade_features/train_trade_na_filled.csv',index=False)
test_trade.to_csv('Trade_features/test_trade_na_filled.csv',index=False)

NameError: name 'training_trade' is not defined

In [ ]:
## 2. 시계열 정보 반영